In [1]:
import asyncio
import nest_asyncio
nest_asyncio.apply()

import pandas as pd
from typing import List, Dict, Any, Optional, Tuple, Union

from lib.utils import get_polygon_root, get_93, get_polygon_key, get_nyse_date_tups, estimate_time
from lib.fetcher import HttpRequestFetcher, BatchRequestExecutor
from lib.polygon import make_urls

In [2]:
base_path = get_polygon_root()
api_key = get_polygon_key()
tickers = get_93()

In [3]:
tickers_ = tickers
start_date = '2011-07-01'
end_date = '2021-06-30'
tups = get_nyse_date_tups(start_date, end_date)

urls = make_urls(tickers_, tups, api_key)

In [4]:
len(urls)

233988

In [5]:
urls[0]

'https://api.polygon.io/v2/aggs/ticker/AAPL/range/1/minute/2011-07-01 09:30:00/2011-07-01 16:00:00?adjusted=true&sort=asc&limit=1000&apiKey=nBHpDZaNGDNb5EuRL3HNUGNU_Z2NnD9h'

In [9]:
estimate_time(len(urls), rps=1000, req_time=0.1)

Estimated time for 233988 requests @ 2s per API call: 00:07:47
